In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
data = datasets.load_diabetes(as_frame=True, return_X_y=False)
X = data.data
y = data.target
X['gender'] = pd.cut(X["sex"], bins=[-1,0,1], labels=[0,1])
X = X.drop(['sex'], axis=1)

In [2]:
data.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - age     age in years\n      - sex\n      - bmi     body mass index\n      - bp      average blood pressure\n      - s1      tc, total serum cholesterol\n      - s2      ldl, low-density lipoproteins\n      - s3      hdl, high-density lipoproteins\n      - s4      tch, total cholesterol / HDL\n      - s5      ltg, possibly log of serum triglycerides level\n      - s6      glu, blood sugar

## Big Picture

Given past diabetes data, we want to find out if a person is at risk of diabetes based on certain input features
Model we create will be used by technicians to find the risk of diabetes
Since we have the diabetest dataset, this will be a supervised learning problem. Will also explore unsupervised in the end (Nearest Neighbors with Kernel)
We will start this off as a multiple regression problem. 

While working on this as a regression problem, we will use RMSE for measuriing model performance
Will switch to F1 score when we tackle this as a binary classification problem

## Set aside the test set
Lets set aside some test data set to check our final model performance. Done before we start exporing the data and using it for training.
Question now is, how much do we set aside for test set? Since diabetes dataset has only 442 records, we need to set aside a decent % for test set to be able to find True error. So we will set aside 30%

In [3]:
#hyperparameter
test_ratio = 0.3

### option 1 is to randomly split data
But the issue is that our training set data will change evry time we run our try training the model. Eventually all the data will be part of training so defeats the purpose of have a test set that generalizes true error

In [4]:
# returns a shuffled out of order array of len(X) elements
shuffled_indices = np.random.permutation(len(X))
test_set_size = int(len(X)*test_ratio)

test_indices = shuffled_indices[:test_set_size]
train_indices = shuffled_indices[test_set_size:]
train_X = X.iloc[train_indices]
train_y = y[train_indices]
test_X = X.iloc[test_indices]
test_y = y[test_indices]

In [5]:
len(train_X), len(train_y), len(test_X), len(test_y)

(310, 310, 132, 132)

### option 2 is to use sklearn's split capability
But the issues with option 1 remain even if we set the random_state parameter

In [6]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = test_ratio, random_state=42)

In [7]:
len(train_X), len(train_y), len(test_X), len(test_y)

(309, 309, 133, 133)

### option 3 is the split so that each subgroup is represented well in test set 
diabetes dataset has sex as an attribute that we can possibly use to define subgroups that need representation in test set

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
data = pd.concat([X,y], axis=1)
split = StratifiedShuffleSplit(n_splits=1, test_size = test_ratio, random_state=42)
for train_index, test_index in split.split(data, data["gender"]):
    train_set = data.iloc[train_index]
    test_set = data.iloc[test_index]

In [9]:
len(train_set), len(test_set)

(309, 133)

## Feature engineering
All the columns here are numerical and already normalized (centered around zero)
So we don't have the task of converting non-numerical columns into one-hot-encoding or scaling them

In [10]:
data[['s1','s2','s3','s4','s5','s6']].corr()

,s1,s2,s3,s4,s5,s6
s1,1.000000,0.896663,0.051519,0.542207,0.515501,0.325717
s2,0.896663,1.000000,-0.196455,0.659817,0.318353,0.290600
s3,0.051519,-0.196455,1.000000,-0.738493,-0.398577,-0.273697
s4,0.542207,0.659817,-0.738493,1.000000,0.617857,0.417212
s5,0.515501,0.318353,-0.398577,0.617857,1.000000,0.464670
s6,0.325717,0.290600,-0.273697,0.417212,0.464670,1.000000


## Data Clearning

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   age     442 non-null    float64 
 1   bmi     442 non-null    float64 
 2   bp      442 non-null    float64 
 3   s1      442 non-null    float64 
 4   s2      442 non-null    float64 
 5   s3      442 non-null    float64 
 6   s4      442 non-null    float64 
 7   s5      442 non-null    float64 
 8   s6      442 non-null    float64 
 9   gender  442 non-null    category
 10  target  442 non-null    float64 
dtypes: category(1), float64(10)
memory usage: 35.2 KB


In [12]:
data.groupby('s4').describe()

age                                                              \
          count      mean       std       min       25%       50%       75%   
s4                                                                            
-0.076395  28.0 -0.019656  0.052977 -0.107226 -0.068176 -0.012780  0.022638   
-0.070859   1.0  0.009016       NaN  0.009016  0.009016  0.009016  0.009016   
-0.069383   1.0  0.001751       NaN  0.001751  0.001751  0.001751  0.001751   
-0.053516   1.0  0.048974       NaN  0.048974  0.048974  0.048974  0.048974   
-0.051671   1.0 -0.020045       NaN -0.020045 -0.020045 -0.020045 -0.020045   
...         ...       ...       ...       ...       ...       ...       ...   
 0.130252   1.0 -0.027310       NaN -0.027310 -0.027310 -0.027310 -0.027310   
 0.141322   1.0  0.096197       NaN  0.096197  0.096197  0.096197  0.096197   
 0.145012   2.0 -0.003698  0.023117 -0.020045 -0.011871 -0.003698  0.004475   
 0.155345   1.0  0.023546       NaN  0.023546  0.023546  0.023546  0.023546   
 0.185234   1.0  0.005383       NaN  0.005383  0.005383  0.005383  0.005383   

                      bmi            ...        s6           target  \
                max count      mean  ...       75%       max  count   
s4                                   ...                              
-0.076395  0.067136  28.0 -0.043313  ...  0.012384  0.073480   28.0   
-0.070859  0.009016   1.0 -0.022373  ... -0.038357 -0.038357    1.0   
-0.069383  0.001751   1.0 -0.046085  ... -0.079778 -0.079778    1.0   
-0.053516  0.048974   1.0 -0.030996  ...  0.019633  0.019633    1.0   
-0.051671 -0.020045   1.0 -0.084886  ... -0.046641 -0.046641    1.0   
...             ...   ...       ...  ...       ...       ...    ...   
 0.130252 -0.027310   1.0  0.047685  ...  0.131470  0.131470    1.0   
 0.141322  0.096197   1.0  0.051996  ...  0.061054  0.061054    1.0   
 0.145012  0.012648   2.0  0.060618  ...  0.051734  0.052770    2.0   
 0.155345  0.023546   1.0  0.061696  ...  0.081764  0.081764    1.0   
 0.185234  0.005383   1.0  0.034751  ...  0.073480  0.073480    1.0   

                                                                       
                 mean        std    min     25%    50%     75%    max  
s4                                                                     
-0.076395   97.857143  53.670265   37.0   60.75   89.5  111.25  283.0  
-0.070859   84.000000        NaN   84.0   84.00   84.0   84.00   84.0  
-0.069383  114.000000        NaN  114.0  114.00  114.0  114.00  114.0  
-0.053516  102.000000        NaN  102.0  102.00  102.0  102.00  102.0  
-0.051671   90.000000        NaN   90.0   90.00   90.0   90.00   90.0  
...               ...        ...    ...     ...    ...     ...    ...  
 0.130252  317.000000        NaN  317.0  317.00  317.0  317.00  317.0  
 0.141322  230.000000        NaN  230.0  230.00  230.0  230.00  230.0  
 0.145012  277.000000  41.012193  248.0  262.50  277.0  291.50  306.0  
 0.155345  242.000000        NaN  242.0  242.00  242.0  242.00  242.0  
 0.185234   84.000000        NaN   84.0   84.00   84.0   84.00   84.0  

[66 rows x 72 columns]

### There seems to be some "floor" applied to the s4 value. I am not sure if there is a natural minimum of this parameter or this is just a data set issue
So will ignore this for now

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score


In [14]:
train_X, train_y = train_set.drop("target", axis=1), train_set['target']
test_X, test_y = test_set.drop("target", axis=1), test_set['target']

### Lets try out some regressors and see which one have the most promise
We will then further work on the most promissing models

In [15]:
linear_reg = LinearRegression()
linear_reg.fit(train_X, train_y)
lin_scores = cross_val_score(linear_reg, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
lin_rmse = np.sqrt(-lin_scores)
np.mean(lin_rmse), np.std(lin_rmse)

(53.54012993960013, 5.022513371288058)

In [16]:
svr = SVR()
svr.fit(train_X, train_y)
svr_scores = cross_val_score(svr, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
svr_rmse = np.sqrt(-svr_scores)
np.mean(svr_rmse), np.std(svr_rmse)

(78.52115056808245, 3.4678949953942144)

In [17]:
linear_svr = LinearSVR()
linear_svr.fit(train_X, train_y)
lin_svr_scores = cross_val_score(linear_svr, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
lin_svr_rmse = np.sqrt(-lin_svr_scores)
np.mean(lin_svr_rmse), np.std(lin_svr_rmse)

(93.46606353559879, 9.676018255275071)

In [18]:
poly_svr = SVR(kernel="poly")
poly_svr.fit(train_X, train_y)
poly_svr_scores = cross_val_score(poly_svr, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
poly_svr_rmse = np.sqrt(-poly_svr_scores)
np.mean(poly_svr_rmse), np.std(poly_svr_rmse)

(75.26613152087876, 3.0538237785038684)

In [19]:
dtr = DecisionTreeRegressor()
dtr.fit(train_X, train_y)
dtr_scores = cross_val_score(dtr, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
dtr_rmse = np.sqrt(-dtr_scores)
np.mean(dtr_rmse), np.std(dtr_rmse)

(78.69991754768655, 13.158233430806582)

In [20]:
rfr = RandomForestRegressor()
rfr.fit(train_X, train_y)
rfr_scores = cross_val_score(rfr, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
rfr_rmse = np.sqrt(-rfr_scores)
np.mean(rfr_rmse), np.std(rfr_rmse)

(55.57164469480931, 4.6779219400619265)

In [21]:
knn = KNeighborsRegressor()
knn_scores = cross_val_score(knn, train_X, train_y, scoring="neg_mean_squared_error", cv=10)
knn_rmse = np.sqrt(-knn_scores)
np.mean(knn_rmse), np.std(knn_rmse)

(58.391390015957484, 5.517313703149795)

### fine tune promissing models. 
KNN, Random Forest, and Linear Regressor seem to have the lowest mean RMSE based on cross valdiation

In [22]:
param_grid = [
    {'n_estimators': [1,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,4,6,8]}
]
grid = GridSearchCV(rfr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid.fit(train_X, train_y), grid.best_estimator_, np.sqrt(-grid.best_score_)

(GridSearchCV(cv=5, estimator=RandomForestRegressor(),
              param_grid=[{'max_features': [2, 4, 6, 8],
                           'n_estimators': [1, 10, 30]},
                          {'bootstrap': [False], 'max_features': [2, 4, 6, 8],
                           'n_estimators': [3, 10]}],
              return_train_score=True, scoring='neg_mean_squared_error'),
 RandomForestRegressor(max_features=8, n_estimators=30),
 55.12587151532169)

In [23]:
param_grid = [
    {'n_neighbors':[5,10,20], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
]
grid = GridSearchCV(knn, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid.fit(train_X, train_y), grid.best_estimator_, np.sqrt(-grid.best_score_)

(GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
              param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                         'brute'],
                           'n_neighbors': [5, 10, 20]}],
              return_train_score=True, scoring='neg_mean_squared_error'),
 KNeighborsRegressor(n_neighbors=10),
 55.84335145895315)

### no params for Linear Reg
So we will try Ridge and Lasso regression with various alpha values

In [24]:
from sklearn.linear_model import Ridge
param_grid = [{'alpha':[0.0001, 0.001, 0.01,0.1,1]}]
ridge = Ridge()
grid = GridSearchCV(ridge, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid.fit(train_X, train_y), grid.best_estimator_, np.sqrt(-grid.best_score_), grid.best_estimator_.coef_

(GridSearchCV(cv=5, estimator=Ridge(),
              param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}],
              return_train_score=True, scoring='neg_mean_squared_error'),
 Ridge(alpha=0.0001),
 53.88554602191953,
 array([  -9.27893714,  485.20515584,  398.46438269, -909.72655032,
         590.48360399,  105.34716445,  192.12083402,  891.35461304,
         -19.09629896,  -23.16317879]))

In [25]:
from sklearn.linear_model import Lasso
lasso = Lasso()
grid = GridSearchCV(lasso, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid.fit(train_X, train_y), grid.best_estimator_, np.sqrt(-grid.best_score_), grid.best_estimator_.coef_

(GridSearchCV(cv=5, estimator=Lasso(),
              param_grid=[{'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}],
              return_train_score=True, scoring='neg_mean_squared_error'),
 Lasso(alpha=0.0001),
 53.885810861435004,
 array([  -9.3751195 ,  485.06153318,  398.5557778 , -919.85203406,
         599.00981053,  109.52259889,  192.6452401 ,  895.3321561 ,
         -19.16910918,  -23.16339489]))

## Test Set Results
As of now it lools like Ridge/Lasso regressor may be the one with best result. lets evaluate it with our test set

In [27]:
from sklearn.metrics import mean_squared_error
ridge = Ridge(alpha=0.0001)
ridge.fit(train_X, train_y)
prediction = ridge.predict(test_X)
final_rmse = np.sqrt(mean_squared_error(test_y, prediction))
print(final_rmse)

57.20616862110966


## Observation
There seems to be significant deviation between the train RMSE (3.88554602191953) and test RMSE (57.20616862110966). Which means that we have an issue with variance. We may need a better model or need to train this model longer to better fit the data. BUt there is no "train longer" option for Linear Regression